In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import os
from ccfj import GetStationPairs
from geopy.distance import great_circle
#import shapefile
import geopandas as gp
import yaml
import math

from scipy.spatial import ConvexHull
from scipy.interpolate import griddata
from shapely.geometry import Point, Polygon

from pykrige.ok import OrdinaryKriging


In [2]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import plotlib

In [3]:
flag_project = 1 # 0--regular; 1--repartrition

In [4]:
if flag_project == 0:
    file_project = 'a-project.yml'
elif flag_project == 1:
    file_project = 'a-project_repar.yml'
elif flag_project == 2:
    file_project = 'a-project_voro.yml'
    
with open(file_project, 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']

#name_project = 'project/output_FJSJ_16-01/'               
#name_project = 'project_repartrition/repartrition_01-03/'               
#name_project = 'project_voronoi/voronoi_01-03/'         

In [5]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_repartition_v4.0/output_repar_v9.2_01-01/


In [6]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [7]:
dir_model_fund = dir_project + info_basic['dir_model_fund']
dir_model = dir_project + info_basic['dir_model']
dir_image = dir_project + info_basic['dir_image']+'Vs_compare/'
if not os.path.exists(dir_image):
    os.makedirs(dir_image)
dir_inv_dispernet = dir_project + info_basic['dir_inv_dispernet']
dir_partition = dir_project + info_basic['dir_partition']
key_subworks = info_basic['key_subworks']
M = len(key_subworks)
key_subworks

['1--08-03',
 '2--09-03',
 '3--10-03',
 '4--11-03',
 '5--12-03',
 '6--13-03',
 '7--14-03',
 '8--15-03',
 '9--16-03',
 '10--18-03',
 '12--20-03',
 '13--21-03',
 '15--23-03',
 '17--25-03',
 '18--26-03',
 '19--27-03',
 '20--28-03',
 '21--29-03',
 '22--30-03',
 '23--31-03',
 '25--33-03',
 '26--34-03',
 '28--36-03',
 '30--41-03',
 '31--42-03',
 '33--46-03',
 '34--47-03',
 '35--48-03',
 '36--52-03',
 '37--53-03',
 '38--07-04',
 '39--08-04',
 '40--09-04',
 '41--10-04',
 '42--12-04',
 '43--13-04',
 '44--17-04',
 '45--20-04',
 '46--21-04',
 '48--23-04',
 '49--24-04',
 '50--25-04',
 '51--27-04',
 '52--28-04',
 '53--29-04',
 '54--30-04',
 '55--31-04',
 '56--32-04',
 '57--33-04',
 '58--34-04',
 '59--35-04',
 '60--36-04',
 '63--47-04',
 '65--49-04',
 '66--52-04',
 '67--53-04',
 '68--08-05',
 '69--12-05',
 '70--13-05',
 '71--14-05',
 '72--16-05',
 '75--22-05',
 '76--24-05',
 '77--26-05',
 '78--27-05',
 '79--28-05',
 '80--29-05',
 '81--30-05',
 '84--35-05',
 '85--36-05',
 '86--37-05',
 '88--39-05',
 

In [8]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist() 
lat_stations_all =  stainfo['latitude'].tolist() 
lon_stations_all =  stainfo['longitude'].tolist() 

In [9]:
stations_partition = {}
lat_stations_partition = {}
lon_stations_partition = {}
lat_centroid_partition = []
lon_centroid_partition = []
num_stations = []
for key in key_subworks:
    filepath = dir_partition + str(key) + '.txt'
    stations_this, lat_stations_this, lon_stations_this = np.loadtxt(filepath, dtype='str' , unpack=True)
    stations_partition[key] = stations_this
    lat_stations_partition[key] = lat_stations_this.astype(float)
    lon_stations_partition[key] = lon_stations_this.astype(float)
    num_stations.append(len(stations_this))
    lat_centroid_partition.append(np.mean(lat_stations_this.astype(float)))
    lon_centroid_partition.append(np.mean(lon_stations_this.astype(float)))

In [10]:
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

### Interpolate 3D structure

In [11]:
struc = {}
struc_fund = {}
flag = 0
for key_subwork in key_subworks:
    file_model = dir_model + 'model_'+key_subwork+'.txt'
    file_model_fund = dir_model_fund + 'model_'+key_subwork+'.txt'
    model = np.loadtxt(file_model)
    model_fund = np.loadtxt(file_model_fund)
    struc[key_subwork] = {}
    struc[key_subwork]['layer'] = model[:, 0]
    struc[key_subwork]['z'] = model[:, 1]
    struc[key_subwork]['rho'] = model[:, 2]
    struc[key_subwork]['vs'] = model[:, 3]
    struc[key_subwork]['vp'] = model[:, 4]
    struc[key_subwork]['std'] = model[:, 5]
    struc_fund[key_subwork] = {}
    struc_fund[key_subwork]['layer'] = model_fund[:, 0]
    struc_fund[key_subwork]['z'] = model_fund[:, 1]
    struc_fund[key_subwork]['rho'] = model_fund[:, 2]
    struc_fund[key_subwork]['vs'] = model_fund[:, 3]
    struc_fund[key_subwork]['vp'] = model_fund[:, 4]
    struc_fund[key_subwork]['std'] = model_fund[:, 5]
    flag += 1
    print('Read structure model: %s (%d/%d)' % (key_subwork, flag, M))

vs_horizon = {}
vs_horizon_fund = {}
dz = 5
N = 40
for i in range(N):
    vs_horizon[i] = []
    vs_horizon_fund[i] = []
    for key_subwork in key_subworks:
        vs_horizon[i].append(struc[key_subwork]['vs'][i])
        vs_horizon_fund[i].append(struc_fund[key_subwork]['vs'][i])

Read structure model: 1--08-03 (1/466)
Read structure model: 2--09-03 (2/466)
Read structure model: 3--10-03 (3/466)
Read structure model: 4--11-03 (4/466)
Read structure model: 5--12-03 (5/466)
Read structure model: 6--13-03 (6/466)
Read structure model: 7--14-03 (7/466)
Read structure model: 8--15-03 (8/466)
Read structure model: 9--16-03 (9/466)
Read structure model: 10--18-03 (10/466)
Read structure model: 12--20-03 (11/466)
Read structure model: 13--21-03 (12/466)
Read structure model: 15--23-03 (13/466)
Read structure model: 17--25-03 (14/466)
Read structure model: 18--26-03 (15/466)
Read structure model: 19--27-03 (16/466)
Read structure model: 20--28-03 (17/466)
Read structure model: 21--29-03 (18/466)
Read structure model: 22--30-03 (19/466)
Read structure model: 23--31-03 (20/466)
Read structure model: 25--33-03 (21/466)
Read structure model: 26--34-03 (22/466)
Read structure model: 28--36-03 (23/466)
Read structure model: 30--41-03 (24/466)
Read structure model: 31--42-03 (2

In [12]:
num_lons = 100
num_lats = 100
dz = 5
N = 40
type_interp = 'linear'

v_min = 0.3
v_max = 0.65

In [13]:
# generate grid
xx = np.linspace(np.min(lon_centroid_partition), np.max(lon_centroid_partition), num_lons)
yy = np.linspace(np.min(lat_centroid_partition), np.max(lat_centroid_partition), num_lats)
A,B = np.meshgrid(xx, yy)
X_star = np.hstack((A.flatten()[:,None], B.flatten()[:,None]))
lon_grid = X_star[:,0]
lat_grid = X_star[:,1]
# cut
points = np.column_stack((lon_centroid_partition, lat_centroid_partition))
hull = ConvexHull(points)
polygon = Polygon(points[hull.vertices])
index = []
lon_inter_in = []
lat_inter_in = []
for i in range(len(lon_grid)):
    if polygon.contains(Point(lon_grid[i], lat_grid[i])):
        index.append(i)
        lon_inter_in.append(lon_grid[i])
        lat_inter_in.append(lat_grid[i])

In [14]:
vs_inter_horizon_all = {}
vs_inter_fund_horizon_all = {}
vs_inter_fund_horizon_in = {}
vs_inter_horizon_in = {}
for i in range(N):
    print('Interpolating horizon %d/%d' % (i+1,N))
    OK = OrdinaryKriging(lon_centroid_partition, lat_centroid_partition, vs_horizon[i], variogram_model=type_interp,nlags=3)
    zz,ss = OK.execute('grid', xx, yy)
    vs_inter_horizon_all[i] = zz.reshape(len(xx)*len(yy))
    vs_inter_horizon_in[i] = zz.reshape(len(xx)*len(yy))[index]
    OK = OrdinaryKriging(lon_centroid_partition, lat_centroid_partition, vs_horizon_fund[i], variogram_model=type_interp,nlags=3)
    zz,ss = OK.execute('grid', xx, yy)
    vs_inter_fund_horizon_all[i] = zz.reshape(len(xx)*len(yy))
    vs_inter_fund_horizon_in[i] = zz.reshape(len(xx)*len(yy))[index]

Interpolating horizon 1/40
Interpolating horizon 2/40
Interpolating horizon 3/40
Interpolating horizon 4/40
Interpolating horizon 5/40
Interpolating horizon 6/40
Interpolating horizon 7/40
Interpolating horizon 8/40
Interpolating horizon 9/40
Interpolating horizon 10/40
Interpolating horizon 11/40
Interpolating horizon 12/40
Interpolating horizon 13/40
Interpolating horizon 14/40
Interpolating horizon 15/40
Interpolating horizon 16/40
Interpolating horizon 17/40
Interpolating horizon 18/40
Interpolating horizon 19/40
Interpolating horizon 20/40
Interpolating horizon 21/40
Interpolating horizon 22/40
Interpolating horizon 23/40
Interpolating horizon 24/40
Interpolating horizon 25/40
Interpolating horizon 26/40
Interpolating horizon 27/40
Interpolating horizon 28/40
Interpolating horizon 29/40
Interpolating horizon 30/40
Interpolating horizon 31/40
Interpolating horizon 32/40
Interpolating horizon 33/40
Interpolating horizon 34/40
Interpolating horizon 35/40
Interpolating horizon 36/40
I

In [15]:
for i in range(N):
    print('Plot horizon %d/%d' % (i+1,N))
    fig = plt.figure(figsize=(20, 14))
    ax = fig.add_subplot(221)
    ax.scatter(lon_stations_all, lat_stations_all,marker='.',color='k')
    #ax.set_xticks([])  #去掉横坐标值
    #ax.set_yticks([])  #去掉纵坐标值
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Vs at depth '+str((i+1)*dz)+' Fundamental')
    flag_add = 0.0005
    ax.set_xlim([min(lon_stations_all)-flag_add, max(lon_stations_all)+flag_add])
    ax.set_ylim([min(lat_stations_all)-flag_add, max(lat_stations_all)+flag_add])
    for j in range(len(faults)):
        ax.plot(faults['clark'+str(j+1)]['lon'], faults['clark'+str(j+1)]['lat'], 'k')
    im = ax.scatter(lon_centroid_partition,lat_centroid_partition,c=vs_horizon_fund[i],cmap='gist_rainbow',s=100)
    plt.colorbar(im)
    ax.set_title('Vs at depth '+str((i+1)*dz)+' Fundamental')
    ax.set_xticks([])  #去掉横坐标值
    ax.set_yticks([])  #去掉纵坐标值

    ax = fig.add_subplot(222)
    ax.scatter(lon_stations_all, lat_stations_all,marker='.',color='k')
    for j in range(len(faults)):
        ax.plot(faults['clark'+str(j+1)]['lon'], faults['clark'+str(j+1)]['lat'], 'k')
    #im = plt.scatter(lon_inter_in, lat_inter_in, s=10, c=vs_inter_horizon_in[i], cmap='gist_rainbow', vmin=v_min, vmax=v_max)
    im = plt.scatter(lon_inter_in, lat_inter_in, s=10, c=vs_inter_fund_horizon_in[i], cmap='gist_rainbow')
    plt.colorbar(im)
    ax.set_xticks([])  #去掉横坐标值
    ax.set_yticks([])  #去掉纵坐标值
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Vs at depth '+str((i+1)*dz)+' Fundamental')
    

    ax = fig.add_subplot(223)
    ax.scatter(lon_stations_all, lat_stations_all,marker='.',color='k')
    #ax.set_xticks([])  #去掉横坐标值
    #ax.set_yticks([])  #去掉纵坐标值
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_title('Vs at depth '+str((i+1)*dz)+' Fundamental')
    flag_add = 0.0005
    ax.set_xlim([min(lon_stations_all)-flag_add, max(lon_stations_all)+flag_add])
    ax.set_ylim([min(lat_stations_all)-flag_add, max(lat_stations_all)+flag_add])
    for j in range(len(faults)):
        ax.plot(faults['clark'+str(j+1)]['lon'], faults['clark'+str(j+1)]['lat'], 'k')
    im = ax.scatter(lon_centroid_partition,lat_centroid_partition,c=vs_horizon[i],cmap='gist_rainbow',s=100)
    plt.colorbar(im)
    ax.set_title('Vs at depth '+str((i+1)*dz)+' Overtones')
    ax.set_xticks([])  #去掉横坐标值
    ax.set_yticks([])  #去掉纵坐标值

    ax = fig.add_subplot(224)
    ax.scatter(lon_stations_all, lat_stations_all,marker='.',color='k')
    for j in range(len(faults)):
        ax.plot(faults['clark'+str(j+1)]['lon'], faults['clark'+str(j+1)]['lat'], 'k')
    #im = plt.scatter(lon_inter_in, lat_inter_in, s=10, c=vs_inter_horizon_in[i], cmap='gist_rainbow', vmin=v_min, vmax=v_max)
    im = plt.scatter(lon_inter_in, lat_inter_in, s=10, c=vs_inter_horizon_in[i], cmap='gist_rainbow')
    plt.colorbar(im)
    ax.set_xticks([])  #去掉横坐标值
    ax.set_yticks([])  #去掉纵坐标值
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    plt.tight_layout()
    ax.set_title('Vs at depth '+str((i+1)*dz)+' Overtones')
    plt.savefig(dir_image+'Vs_hori_'+str((i+1)*dz)+'m.png',dpi=100)
    plt.close()

Plot horizon 1/40
Plot horizon 2/40
Plot horizon 3/40
Plot horizon 4/40
Plot horizon 5/40
Plot horizon 6/40
Plot horizon 7/40
Plot horizon 8/40
Plot horizon 9/40
Plot horizon 10/40
Plot horizon 11/40
Plot horizon 12/40
Plot horizon 13/40
Plot horizon 14/40
Plot horizon 15/40
Plot horizon 16/40
Plot horizon 17/40
Plot horizon 18/40
Plot horizon 19/40
Plot horizon 20/40
Plot horizon 21/40
Plot horizon 22/40
Plot horizon 23/40
Plot horizon 24/40
Plot horizon 25/40
Plot horizon 26/40
Plot horizon 27/40
Plot horizon 28/40
Plot horizon 29/40
Plot horizon 30/40
Plot horizon 31/40
Plot horizon 32/40
Plot horizon 33/40
Plot horizon 34/40
Plot horizon 35/40
Plot horizon 36/40
Plot horizon 37/40
Plot horizon 38/40
Plot horizon 39/40
Plot horizon 40/40
